In [1]:
!rm -rf ./logs/ 
%run Model.ipynb
%run dataset_loader.ipynb
import tensorflow as tf
#from dataset_loader import load_dataset

test_no = 15

#tf.enable_eager_execution()
INPUT_WIDTH = 64
INPUT_HEIGHT = 64

INPUT_SIZE = 64

INPUT_CHANNELS = 1

NUM_CLASSES = 5

LEARNING_RATE = 0.005   # Original value: 0.01
DROPOUT_RATE = 0.05

EPOCHS = 1000
GLOBAL_STEPS = 100
BATCH_SIZE = 128

NUM_PARRALLEL_THREADS = 12

GRAYSCALE=True
IS_FREQUENCY = False


train_sub_dir = '/data/small_dataset/image_format_small_dataset/*/*'
val_sub_dir = '/data/small_dataset/image_format_small_dataset_val/*/*'
test_sub_dir = '/data/small_dataset/image_format_small_dataset_test/*/*'


# train_sub_dir = '/data/medium_dataset/medium_dataset_train/*/*'
# val_sub_dir = '/data/medium_dataset/medium_dataset_val/*/*'
# test_sub_dir = '/data/medium_dataset/medium_dataset_test/*/*'



frequency_train_sub_dir = '/data/cogisen_frequency_data/image_format_small_dataset_cogisen_updated/*/*/*.csv'
frequency_val_sub_dir = '/data/cogisen_frequency_data/image_format_small_dataset_val_cogisen_updated/*/*/*.csv'
frequency_test_sub_dir = '/data/cogisen_frequency_data/image_format_small_dataset_test_cogisen_updated/*/*/*.csv'


home_dir = os.path.dirname(os.path.realpath('__file__'))

train_dir = home_dir + train_sub_dir
val_dir = home_dir + val_sub_dir
test_dir = home_dir + test_sub_dir


frequency_train_dir = home_dir + frequency_train_sub_dir
frequency_val_dir = home_dir + frequency_val_sub_dir
frequency_test_dir = home_dir + frequency_test_sub_dir



training_dataset= load_dataset(img_dir= train_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

# logdir = './log/simple_model/frequency_data/grayscale'+str(experiment_no)
# weight_dir = './weights/simple_model/frequency_data/grayscale/'+str(experiment_no)+'/model'


logdir = home_dir+'/log/test/' + str(test_no)

weight_dir = home_dir+'/weights/test/' +str(test_no)

weights = weight_dir +'/model'
if not os.path.exists(weight_dir):
    os.makedirs(os.path.join(weight_dir))
    


/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'XLA_GPU']

In [3]:
get_available_gpus()

['/device:XLA_GPU:0']

In [4]:

handle = tf.placeholder(tf.string, shape=[],name="data_handle")

with tf.name_scope('iterators'):
    training_iterator = tf.data.Iterator.from_structure(training_dataset.output_types,training_dataset.output_shapes)
    training_init_op = training_iterator.make_initializer(training_dataset)
    

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


In [5]:
iterator = tf.data.Iterator.from_string_handle(
    handle, training_iterator.output_types,training_dataset.output_shapes)

next_training_element = iterator.get_next()



In [6]:
#with tf.device('/device:XLA_GPU:0'):
model = Simple_Model(next_training_element,input_height=INPUT_HEIGHT,input_width=INPUT_WIDTH, num_classes=NUM_CLASSES,input_channels=INPUT_CHANNELS,dropout_rate=DROPOUT_RATE,learning_rate=LEARNING_RATE)


In [7]:
with tf.device('/device:XLA_GPU:0'):

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        
        sess.run(tf.global_variables_initializer())
        
        file_writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

        
        summary_operation = tf.summary.merge_all()
        
        saver = tf.train.Saver()
        saver.save(sess, weights)

        for epoch in range(EPOCHS):
            training_string_handle =  sess.run(training_iterator.string_handle())

            sess.run(training_init_op,feed_dict={handle:training_string_handle})
            saver.save(sess, weights,global_step=GLOBAL_STEPS,write_meta_graph=False)
            while True:
                try:
                    _, accuracy,summary,predictions,Y= sess.run([model.training_operation,model.accuracy_operation,summary_operation,model.predictions,model.Y],feed_dict={handle:training_string_handle})
                   
                
                except tf.errors.OutOfRangeError:
                    print("accuracy for training set for epoch {0} is  :  {1}".format(epoch,accuracy))
                    file_writer.add_summary(summary, epoch)                
                    
                    break
                
            
           





accuracy for training set for epoch 0 is  :  0.16304348409175873
accuracy for training set for epoch 1 is  :  0.22826087474822998
accuracy for training set for epoch 2 is  :  0.21739129722118378
accuracy for training set for epoch 3 is  :  0.17391304671764374
accuracy for training set for epoch 4 is  :  0.15217390656471252
accuracy for training set for epoch 5 is  :  0.260869562625885
accuracy for training set for epoch 6 is  :  0.14130434393882751
accuracy for training set for epoch 7 is  :  0.15217390656471252
accuracy for training set for epoch 8 is  :  0.16304348409175873
accuracy for training set for epoch 9 is  :  0.20652173459529877
accuracy for training set for epoch 10 is  :  0.19565217196941376
accuracy for training set for epoch 11 is  :  0.21739129722118378
accuracy for training set for epoch 12 is  :  0.18478260934352875
accuracy for training set for epoch 13 is  :  0.20652173459529877
accuracy for training set for epoch 14 is  :  0.30434781312942505
accuracy for training 

accuracy for training set for epoch 127 is  :  0.1304347813129425
accuracy for training set for epoch 128 is  :  0.20652173459529877
accuracy for training set for epoch 129 is  :  0.30434781312942505
accuracy for training set for epoch 130 is  :  0.22826087474822998
accuracy for training set for epoch 131 is  :  0.19565217196941376
accuracy for training set for epoch 132 is  :  0.239130437374115
accuracy for training set for epoch 133 is  :  0.27173912525177
accuracy for training set for epoch 134 is  :  0.18478260934352875
accuracy for training set for epoch 135 is  :  0.22826087474822998
accuracy for training set for epoch 136 is  :  0.25
accuracy for training set for epoch 137 is  :  0.20652173459529877
accuracy for training set for epoch 138 is  :  0.16304348409175873
accuracy for training set for epoch 139 is  :  0.21739129722118378
accuracy for training set for epoch 140 is  :  0.260869562625885
accuracy for training set for epoch 141 is  :  0.18478260934352875
accuracy for train

accuracy for training set for epoch 252 is  :  0.20652173459529877
accuracy for training set for epoch 253 is  :  0.25
accuracy for training set for epoch 254 is  :  0.20652173459529877
accuracy for training set for epoch 255 is  :  0.25
accuracy for training set for epoch 256 is  :  0.22826087474822998
accuracy for training set for epoch 257 is  :  0.15217390656471252
accuracy for training set for epoch 258 is  :  0.17391304671764374
accuracy for training set for epoch 259 is  :  0.27173912525177
accuracy for training set for epoch 260 is  :  0.19565217196941376
accuracy for training set for epoch 261 is  :  0.15217390656471252
accuracy for training set for epoch 262 is  :  0.19565217196941376
accuracy for training set for epoch 263 is  :  0.20652173459529877
accuracy for training set for epoch 264 is  :  0.17391304671764374
accuracy for training set for epoch 265 is  :  0.18478260934352875
accuracy for training set for epoch 266 is  :  0.22826087474822998
accuracy for training set fo

accuracy for training set for epoch 379 is  :  0.15217390656471252
accuracy for training set for epoch 380 is  :  0.25
accuracy for training set for epoch 381 is  :  0.19565217196941376
accuracy for training set for epoch 382 is  :  0.22826087474822998
accuracy for training set for epoch 383 is  :  0.19565217196941376
accuracy for training set for epoch 384 is  :  0.20652173459529877
accuracy for training set for epoch 385 is  :  0.20652173459529877
accuracy for training set for epoch 386 is  :  0.21739129722118378
accuracy for training set for epoch 387 is  :  0.22826087474822998
accuracy for training set for epoch 388 is  :  0.17391304671764374
accuracy for training set for epoch 389 is  :  0.22826087474822998
accuracy for training set for epoch 390 is  :  0.18478260934352875
accuracy for training set for epoch 391 is  :  0.27173912525177
accuracy for training set for epoch 392 is  :  0.18478260934352875
accuracy for training set for epoch 393 is  :  0.18478260934352875
accuracy for 

accuracy for training set for epoch 505 is  :  0.19565217196941376
accuracy for training set for epoch 506 is  :  0.18478260934352875
accuracy for training set for epoch 507 is  :  0.19565217196941376
accuracy for training set for epoch 508 is  :  0.20652173459529877
accuracy for training set for epoch 509 is  :  0.20652173459529877
accuracy for training set for epoch 510 is  :  0.20652173459529877
accuracy for training set for epoch 511 is  :  0.18478260934352875
accuracy for training set for epoch 512 is  :  0.18478260934352875
accuracy for training set for epoch 513 is  :  0.239130437374115
accuracy for training set for epoch 514 is  :  0.25
accuracy for training set for epoch 515 is  :  0.19565217196941376
accuracy for training set for epoch 516 is  :  0.14130434393882751
accuracy for training set for epoch 517 is  :  0.19565217196941376
accuracy for training set for epoch 518 is  :  0.22826087474822998
accuracy for training set for epoch 519 is  :  0.1195652186870575
accuracy for 

accuracy for training set for epoch 631 is  :  0.25
accuracy for training set for epoch 632 is  :  0.19565217196941376
accuracy for training set for epoch 633 is  :  0.1304347813129425
accuracy for training set for epoch 634 is  :  0.18478260934352875
accuracy for training set for epoch 635 is  :  0.19565217196941376
accuracy for training set for epoch 636 is  :  0.20652173459529877
accuracy for training set for epoch 637 is  :  0.19565217196941376
accuracy for training set for epoch 638 is  :  0.15217390656471252
accuracy for training set for epoch 639 is  :  0.16304348409175873
accuracy for training set for epoch 640 is  :  0.239130437374115
accuracy for training set for epoch 641 is  :  0.18478260934352875
accuracy for training set for epoch 642 is  :  0.20652173459529877
accuracy for training set for epoch 643 is  :  0.20652173459529877
accuracy for training set for epoch 644 is  :  0.239130437374115
accuracy for training set for epoch 645 is  :  0.22826087474822998
accuracy for tr

accuracy for training set for epoch 756 is  :  0.21739129722118378
accuracy for training set for epoch 757 is  :  0.19565217196941376
accuracy for training set for epoch 758 is  :  0.19565217196941376
accuracy for training set for epoch 759 is  :  0.20652173459529877
accuracy for training set for epoch 760 is  :  0.239130437374115
accuracy for training set for epoch 761 is  :  0.25
accuracy for training set for epoch 762 is  :  0.22826087474822998
accuracy for training set for epoch 763 is  :  0.25
accuracy for training set for epoch 764 is  :  0.15217390656471252
accuracy for training set for epoch 765 is  :  0.20652173459529877
accuracy for training set for epoch 766 is  :  0.17391304671764374
accuracy for training set for epoch 767 is  :  0.16304348409175873
accuracy for training set for epoch 768 is  :  0.20652173459529877
accuracy for training set for epoch 769 is  :  0.21739129722118378
accuracy for training set for epoch 770 is  :  0.20652173459529877
accuracy for training set f

accuracy for training set for epoch 883 is  :  0.30434781312942505
accuracy for training set for epoch 884 is  :  0.21739129722118378
accuracy for training set for epoch 885 is  :  0.21739129722118378
accuracy for training set for epoch 886 is  :  0.21739129722118378
accuracy for training set for epoch 887 is  :  0.260869562625885
accuracy for training set for epoch 888 is  :  0.20652173459529877
accuracy for training set for epoch 889 is  :  0.29347825050354004
accuracy for training set for epoch 890 is  :  0.16304348409175873
accuracy for training set for epoch 891 is  :  0.22826087474822998
accuracy for training set for epoch 892 is  :  0.17391304671764374
accuracy for training set for epoch 893 is  :  0.28260868787765503
accuracy for training set for epoch 894 is  :  0.17391304671764374
accuracy for training set for epoch 895 is  :  0.22826087474822998
accuracy for training set for epoch 896 is  :  0.25
accuracy for training set for epoch 897 is  :  0.27173912525177
accuracy for tr

In [8]:
meta_dir = weights+'.meta'



In [9]:
tf.reset_default_graph()
graph = tf.get_default_graph()
saver = tf.train.import_meta_graph(meta_dir)

val_dataset = load_dataset(img_dir= val_dir,frequency_dir=frequency_val_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset,name='val_init_op')
 

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    
    print('validation dataset...')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(weight_dir))
   
    iterator_handle = graph.get_tensor_by_name('data_handle:0')
    val_handle = sess.run(val_iterator.string_handle())
    

    
    sess.run(val_init_op,feed_dict={iterator_handle:val_handle})

    accuracy_op = graph.get_tensor_by_name('accuracy/accuracy_operation:0')
   
    while True:
        try:
            val_accuracy = sess.run([accuracy_op],feed_dict={iterator_handle:val_handle})

        except tf.errors.OutOfRangeError:
            print("accuracy for validation set :  {0}".format(val_accuracy))
            break



validation dataset...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/shubham/Desktop/cogisen/code/weights/test/14/model-100
accuracy for validation set :  [0.20333333]


In [10]:
tf.reset_default_graph()
graph = tf.get_default_graph()
saver = tf.train.import_meta_graph(meta_dir)

test_dataset = load_dataset(img_dir= test_dir,frequency_dir=frequency_test_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,test_dataset.output_shapes)
test_init_op = test_iterator.make_initializer(test_dataset,name='test_init_op')


with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    
    print('testing dataset...')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(weight_dir))
   # val_op = graph.get_operation_by_name("iterators/val_init_op")
    iterator_handle = graph.get_tensor_by_name('data_handle:0')
    test_handle = sess.run(test_iterator.string_handle())

    
    sess.run(test_init_op,feed_dict={iterator_handle:test_handle})

    accuracy_op = graph.get_tensor_by_name('accuracy/accuracy_operation:0')
   
    while True:
        try:
            test_accuracy = sess.run([accuracy_op],feed_dict={iterator_handle:test_handle})

        except tf.errors.OutOfRangeError:
            print("accuracy for testing set :  {0}".format(test_accuracy))
            break



testing dataset...
INFO:tensorflow:Restoring parameters from /home/shubham/Desktop/cogisen/code/weights/test/14/model-100
accuracy for testing set :  [0.20133333]
